In [78]:
import pytesseract
from PIL import Image
import re
import requests
from io import BytesIO

In [52]:
image_path_1 = "notebook/Capture d’écran 2024-10-08 à 11.15.31.png"
extractedInformation_1 = pytesseract.image_to_string(Image.open(image_path_1))
print(extractedInformation_1)

~ LOSEC 20 mg, 1 gélule le matin pendant 1 mois. AR 3 fois, NS

~ LACTIBIANE, 1 sachet dans un verre d'eau le matin pendant 10 jours. NR



In [ ]:
def getImage(source):
    '''
    fonction permettant de reconstruire l'image à partir d'un objet bytes
    :param source: objet bytes
    :return: image dans le format communiqué 
    '''
    response = requests.get(source)
    img = Image.open(BytesIO(response.content))
    return img

In [67]:
image = "notebook/Capture_decran_2024-10-08_a_15.03.39.png"
def lecture_image(image):
    '''
    A FAIRE reconstruction de l'image et extraction des information de l'image
    :param image:encore de l'image recu par l'API 
    :return:un texte des informations capturé dans l'image 
    '''
    extractedInformation = pytesseract.image_to_string(Image.open(image))
    return extractedInformation

extractedInformation = lecture_image(image)

In [71]:
def transforme_list(information):
    '''
    fonction permettant de transformer mon texte d'entrée en une liste d'élement avec comme 
    separteur un saut de ligne
    :param information: le texte de mon image
    :return: une liste de chaques médicaments et ses informations de mon texte
    '''
    liste_elements = [element.strip() for element in information.strip().split('\n\n') if element.strip()]
    return liste_elements

element = transforme_list(extractedInformation)
element

['aie eorer—<e—e',
 'BECLOMETASONE DIPROPIONATE 100 pg/dose susp p pulv nasal (RHINOMAXIL)\n1 43 fois par jour pendant 5 jours',
 'PARACETAMOL 1 g cp (DOLIPRANE)\n1g/6hsidouleurs 2 boites',
 'ALPHA-AMYLASE 200 U CEIP/ml sirop (MAXILASE ALPHA-AMYLASE)\n2 cuilléres a café (10 ml), 3 fois par jour, 5 jours',
 'Signé via Doctolib le 30/04/2024',
 '( 8 Marine Legrand',
 '_ oe']

In [77]:
def extraire_informations(element):
    '''
    fonction permettant d'extraire les informations d'un string qu'on lui donne
    :param element: chaine de caractere
    :return: {
        "Medicament": medicament_nom,
        "nbr_comprimé": nbr_comprime_val,
        "Matin": bool,
        "Midi": bool,
        "Soir": bool,
        "Durée": duree_val
    } 
    '''
    # prend les premiers mots en majuscule de chaque élément avant d'arriver à un int 
    # int qui represente le dosage
    medicament = re.search(r"([A-ZÉÈÊÀÂÙÛÎÔÇ]+(?: [A-ZÉÈÊÀÂÙÛÎÔÇ]+)*)\s+\d", element)
    medicament_nom = medicament.group(1) if medicament else None
    if not medicament_nom:
        return False 
    # prend le nbr de comprimés à prendre pendant le traitement
    nbr_comprime = re.search(r"(\d+)\s+(?:gélule|comprimé|gélules|comprimés|sachet|sachets|cuilléres a café|cuilléres à café|fois|g/)", element, re.IGNORECASE)
    nbr_comprime_val = int(nbr_comprime.group(1)) if nbr_comprime else None
    
    # regarde le moment de la prise de medicament et renvoie True or False
    mots_cle_midi = ["midi", "déjeuner", "jour"]
    matin = "matin" in element.lower() or "petit-déjeuner" in element.lower()
    midi = any(mot in element.lower() for mot in mots_cle_midi)
    soir = "soir" in element.lower() or "diner" in element.lower()
    
    # Extraire la durée entre "pendant" et le point "."
    duree = re.search(r"pendant\s+(\d+)\s+(jours|mois)", element)
    if duree:
        nbr_duree_val = int(duree.group(1))
        type_duree = duree.group(2)
        duree_val = {"nbr_durée": nbr_duree_val, "Durée": type_duree}
    else:
        duree_val = None
    
    return {
        "medicament": medicament_nom,
        "nbr_comprimé": nbr_comprime_val,
        "matin": matin,
        "midi": midi,
        "soir": soir,
        "durée": duree_val
    }

[extraire_informations(element) for element in element if extraire_informations(element) is not False]

[{'medicament': 'BECLOMETASONE DIPROPIONATE',
  'nbr_comprimé': 43,
  'matin': False,
  'midi': True,
  'soir': False,
  'durée': {'nbr_durée': 5, 'Durée': 'jours'}},
 {'medicament': 'PARACETAMOL',
  'nbr_comprimé': None,
  'matin': False,
  'midi': False,
  'soir': False,
  'durée': None},
 {'medicament': 'AMYLASE',
  'nbr_comprimé': 2,
  'matin': False,
  'midi': True,
  'soir': False,
  'durée': None}]